## Dialogue based chatbot

In [1]:
import numpy as np
from typing import Dict, List, Tuple, Optional

## Loading the model

In [2]:
from intent_classification_model import IntentClassificationModel

In [3]:
IntentClassificationModel().predict("payment method k k available cha")

'payment_method'

In [4]:
from entity_recognition import CRFModel

In [5]:
CRFModel().predict("Dharan ma Redmi Note 9 pro 1 ta delivery garidinu")

array(['B-Location', 'O', 'B-Product', 'I-Product', 'I-Product', 'O',
       'B-Quantity', 'O', 'O', 'O'], dtype=object)

In [6]:
products_db = [
            {"id": 1, "name": "IPhone 11", "category": "mobile", "brand": "Apple", "price": 799.99},
            {"id": 2, "name": "Galaxy S21", "category": "mobile", "brand": "Samsung", "price": 699.99},
            {"id": 3, "name": "Sony Television", "category": "TV", "brand": "Sony", "price": 179.99},
            {"id": 5, "name": "Macbook Pro", "category": "laptop", "brand": "Apple", "price": 1299.99},
            {"id": 6, "name": "Redmi Note 9 pro", "category": "mobile", "brand": "Xiaomi", "price": 999.99},
        ]
        

In [7]:
import random
from fuzzywuzzy import process, fuzz

class EcommerceBot:
    def __init__(self, intent_model, entity_model):
        self.intent_model = intent_model
        self.entity_model = entity_model
        
        self.order_db = [
            {
                "Id": "ORD12345",
                "status": "Processing"
            },
            {
                "Id": "ORD00000",
                "status": "Shipping"
            }
        ]
        
        self.intent_handlers = {
            'payment_method': self._handle_payment_method,
            'order_status_inquiry': self._handle_order_status,
            'order_product': self._handle_order_product,
            'product_inquiry': self._handle_product_inquiry
        }
        
        self.products_db = [
            {"id": 1, "name": "IPhone 11", "category": "mobile", "brand": "Apple", "price": 799.99},
            {"id": 2, "name": "Galaxy S21", "category": "mobile", "brand": "Samsung", "price": 699.99},
            {"id": 3, "name": "Sony Television", "category": "TV", "brand": "Sony", "price": 179.99},
            {"id": 5, "name": "Macbook Pro", "category": "laptop", "brand": "Apple", "price": 1299.99},
            {"id": 6, "name": "Redmi Note 9 pro", "category": "mobile", "brand": "Xiaomi", "price": 999.99},
        ]
        
        # keep the context of the last intent and all the entities
        self.conversation_context = {}
        
    def process_message(self, user_message: str) -> str:
        intent = self._predict_intent(user_message)
        entities = self._extract_entities(user_message)
        
        print(f"Intent: {intent}")
        print(f"Entities: {entities}")
        
        # handling the intent
        if intent in self.intent_handlers:
            response = self.intent_handlers[intent](entities)
        else:
            response = "Maaf garnu hola maile hajurko kura bujhna sakina."
            
        return response
    
    def _predict_intent(self, sentence):
        intent = self.intent_model.predict(sentence)
        return intent
    
    def _extract_entities(self, message: str) -> Dict[str, str]:
        entities = self.entity_model.predict(message)
        return self._format_entities(entities, message.split())
    
    def fuzzy_search(self, search_term: str, search_type: str) -> List[Dict]:
        """
        Perform fuzzy search on products based on category, brand, or name
        """
        matches = []
        
        if search_type == 'category':
            # Get unique categories
            categories = set(product['category'] for product in self.products_db)
            matched_category, score = process.extractOne(search_term, categories)
            if score > 80:  # Threshold for matching
                matches = [p for p in self.products_db if p['category'] == matched_category]
                
        elif search_type == 'brand':
            # Get unique brands
            brands = set(product['brand'] for product in self.products_db)
            matched_brand, score = process.extractOne(search_term, brands)
            if score > 80:
                matches = [p for p in self.products_db if p['brand'] == matched_brand]
                
        else:  # Search by product name
            for product in self.products_db:
                score = fuzz.ratio(search_term.lower(), product['name'].lower())
                if score > 60:
                    matches.append(product)
        
        return matches
    
    def format_product_inquiry_message(self, matches: list[dict]):
        message = ""
        for i, product in enumerate(matches):
            if i == 0:
                msg = f"Hami sanga product Rs. {product["price"]} parne {product["name"]} xa\n"
                message += msg
            else:
                msg = f"Arko Hami sanga product {product["name"]} xa ra yaslai Rs. {product["price"]} parcha.\n"
                message += msg
            
        return message
    
    def _format_entities(self, entities: List, sentence: List) -> Dict[str, str]:
        formatted_entities = {
            "Location": None,
            "Product": None,
            "Brand": None,
            "Category": None,
            "Quantity": None,
            "Order_Number": None
        }
        
        for entity, word in zip(entities, sentence):
            # if the entity is present
            if entity in formatted_entities:
                # if it is a single word entity
                formatted_entities[entity] = word
            # if it is a more length word entity
            elif len(entity.split("-")) == 2:
                # check if the entity is the beginning or intermediate
                longer_entity = entity.split("-")
                if longer_entity[0] == "B":
                    formatted_entities[longer_entity[1]] = word
                elif longer_entity[0] == "I":
                    formatted_entities[longer_entity[1]] += " " + word
        # print(formatted_entities)  
        return formatted_entities
    
    def _update_context(self, intent: str, entities: Dict[str, str]):
        self.conversation_context["last_intent"] = intent
        self.conversation_context.update(entities)
        
    def _handle_payment_method(self, entities: Dict[str, str]) -> str:
        response_sample = [
            "Hami esewa, khalti, mobile banking sabei payment method accept garxau.",
            "tapai esewa, khalti ya kunei mobile banking baata payment garna saknu huncha.",
        ]
        return random.sample(response_sample, 1)[0]
    
    def _handle_order_status(self, entities: Dict[str, str]) -> str:
        if entities["Order_Number"] is None:
            return "Kripaya malai hajurko order number dinuhos."
        for orders in self.order_db:
            if orders["Id"] == entities["Order_Number"]:
                status = orders["status"]
                return f"Hajur ko order ko {status} hudei xa."
            
        return "Kripaya sahi order number dinuhos."
    
    def _handle_order_product(self, entities: Dict[str, str]) -> str:
        needed = ["Product", "Quantity", "Location"]
        missing = [entity for entity in needed if entities[entity] is None]
        
        if len(missing) > 0:
            return f"Kripaya pura details dinuhos: {', '.join(missing)} pani vannuhos."
        
        matched_product = self.fuzzy_search(search_term=entities.get("Product"), search_type="name")
        
        if len(matched_product) == 0:
            return f"Maaf garnu hola {entities.get("Product")} available xaina."
        product = matched_product[0]
        
        return f"Tapaiko order confirm vayo {entities["Product"]}, total quantity:  {entities["Quantity"]} wata ra delivery location {entities["Location"]} total Cost Rs. {int(entities["Quantity"]) * product["price"]} hajur ko order number: ORD12345"
    
    def _handle_product_inquiry(self, entities: Dict[str, str]) -> str:
        product = entities.get("Product")
        category = entities.get("Category")
        brand = entities.get("Brand")
        
        if product:
            matches = self.fuzzy_search(search_term=product, search_type="name")
            return self.format_product_inquiry_message(matches)
        elif brand:
            matches = self.fuzzy_search(search_term=brand, search_type="brand")
            return self.format_product_inquiry_message(matches)
        elif category:
            matches = self.fuzzy_search(category, "category")
            print(matches)
            return self.format_product_inquiry_message(matches)
        
        else: 
            return "Kun product ko barema janna chahau huncha name, category wa brand vannu hola"

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
intent_model = IntentClassificationModel()
entity_model = CRFModel()

In [9]:
bot = EcommerceBot(intent_model, entity_model)

In [10]:
if "B-Brand".split("-")[1] in {"Brand": None}:
    print("Hello")

Hello


In [11]:
"A" in {"A": None}

True

In [12]:
bot.process_message("Xiaomi ko Redmi Note 9 mobile ko price kati ho Dharan ma 1 ta delivery garidnu ani ORD12345 track garidnu")

Intent: order_product
Entities: {'Location': 'Dharan', 'Product': 'Redmi Note 9', 'Brand': 'Xiaomi', 'Category': 'mobile', 'Quantity': '1', 'Order_Number': 'ORD12345'}


'Tapaiko order confirm vayo Redmi Note 9, total quantity:  1 wata ra delivery location Dharan total Cost Rs. 999.99 hajur ko order number: ORD12345'

In [13]:
bot.process_message("mero order ko status bujhna man xa")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Kripaya malai hajurko order number dinuhos.'

In [14]:
bot.process_message("mero order id ORD12345 ho")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': 'ORD12345'}


'Hajur ko order ko Processing hudei xa.'

In [15]:
bot.process_message("mero order id ORD11221 ho")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': 'ORD11221'}


'Kripaya sahi order number dinuhos.'

In [16]:
bot.process_message("k k baata pay garna milcha")

Intent: payment_method
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Hami esewa, khalti, mobile banking sabei payment method accept garxau.'

## Fuzzy Search

In [17]:
from fuzzywuzzy import process, fuzz

In [18]:
fuzz.ratio("yo auta text ho", "yo arko text ho")

80

In [19]:
fuzz.partial_ratio("herayo auta test ho", "yo auta test ho")

100

In [20]:
categories = set(product["category"] for product in products_db)
categories

{'TV', 'laptop', 'mobile'}

In [21]:
process.extractOne("laptop", categories)

('laptop', 100)

In [22]:
products_db = [
            {"id": 1, "name": "IPhone 11", "category": "mobile", "brand": "Apple", "price": 799.99},
            {"id": 2, "name": "Galaxy S21", "category": "mobile", "brand": "Samsung", "price": 699.99},
            {"id": 3, "name": "Sony Television", "category": "TV", "brand": "Sony", "price": 179.99},
            {"id": 5, "name": "Macbook Pro", "category": "laptop", "brand": "Apple", "price": 1299.99},
            {"id": 6, "name": "Redmi Note 9 pro", "category": "mobile", "brand": "Xiaomi", "price": 999.99},
        ]
orders_db = [
            {"order_id": "ORD12345", "status": "delivered", "delivery_date": "2024-01-01", "products": ["iPhone 13"]},
            {"order_id": "ORD12100", "status": "in transit", "delivery_date": "2024-01-10", "products": ["Ultra Boost"]},
            {"order_id": "ORD00111", "status": "processing", "delivery_date": None, "products": ["Macbook Pro"]},
        ]

In [23]:
def fuzzy_search(search_term: str, search_type: str) -> List[Dict]:
        """
        Perform fuzzy search on products based on category, brand, or name
        """
        matches = []
        
        if search_type == 'category':
            # Get unique categories
            categories = set(product['category'] for product in products_db)
            matched_category, score = process.extractOne(search_term, categories)
            if score > 80:  # Threshold for matching
                matches = [p for p in products_db if p['category'] == matched_category]
                
        elif search_type == 'brand':
            # Get unique brands
            brands = set(product['brand'] for product in products_db)
            matched_brand, score = process.extractOne(search_term, brands)
            if score > 80:
                matches = [p for p in products_db if p['brand'] == matched_brand]
                
        else:  # Search by product name
            for product in products_db:
                score = fuzz.ratio(search_term.lower(), product['name'].lower())
                if score > 60:
                    matches.append(product)
        
        return matches

In [24]:
fuzzy_search(search_term="Laptop", search_type="category")

[{'id': 5,
  'name': 'Macbook Pro',
  'category': 'laptop',
  'brand': 'Apple',
  'price': 1299.99}]

In [25]:
natch_product = fuzzy_search(search_term="galaxy", search_type="name")

In [26]:
for product in natch_product:
    print(f"Hami sanga {product["name"]} price rs. {product["price"]}")

Hami sanga Galaxy S21 price rs. 699.99


In [27]:
def format_product_inquiry_message(matches: list[dict]):
    message = ""
    for i, product in enumerate(matches):
        if i == 0:
            msg = f"Hami sanga product Rs. {product["price"]} parne {product["name"]} xa\n"
            message += msg
        else:
            msg = f"Arko Hami sanga product {product["name"]} xa ra yaslai Rs. {product["price"]} parcha.\n"
            message += msg
            
    return message

In [28]:
format_product_inquiry_message([{'id': 5, 'name': 'Macbook Pro', 'category': 'laptop', 'brand': 'Apple', 'price': 1299.99}])

'Hami sanga product Rs. 1299.99 parne Macbook Pro xa\n'

In [29]:
response = format_product_inquiry_message(fuzzy_search(search_term="mobile", search_type="category"))

In [30]:
fuzzy_search("mobile", "category")

[{'id': 1,
  'name': 'IPhone 11',
  'category': 'mobile',
  'brand': 'Apple',
  'price': 799.99},
 {'id': 2,
  'name': 'Galaxy S21',
  'category': 'mobile',
  'brand': 'Samsung',
  'price': 699.99},
 {'id': 6,
  'name': 'Redmi Note 9 pro',
  'category': 'mobile',
  'brand': 'Xiaomi',
  'price': 999.99}]

In [31]:
print(response)

Hami sanga product Rs. 799.99 parne IPhone 11 xa
Arko Hami sanga product Galaxy S21 xa ra yaslai Rs. 699.99 parcha.
Arko Hami sanga product Redmi Note 9 pro xa ra yaslai Rs. 999.99 parcha.



In [32]:
bot = EcommerceBot(intent_model,entity_model)

In [33]:
bot.process_message("kun kun product available xa")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Kun product ko barema janna chahau huncha name, category wa brand vannu hola'

In [34]:
bot.process_message("malai laptop kun kun xa vanidinu")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': 'laptop', 'Quantity': None, 'Order_Number': None}
[{'id': 5, 'name': 'Macbook Pro', 'category': 'laptop', 'brand': 'Apple', 'price': 1299.99}]


'Hami sanga product Rs. 1299.99 parne Macbook Pro xa\n'

In [35]:
print(bot.process_message("malai mobile kun kun xa jankari dinu"))

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': 'mobile', 'Quantity': None, 'Order_Number': None}
[{'id': 1, 'name': 'IPhone 11', 'category': 'mobile', 'brand': 'Apple', 'price': 799.99}, {'id': 2, 'name': 'Galaxy S21', 'category': 'mobile', 'brand': 'Samsung', 'price': 699.99}, {'id': 6, 'name': 'Redmi Note 9 pro', 'category': 'mobile', 'brand': 'Xiaomi', 'price': 999.99}]
Hami sanga product Rs. 799.99 parne IPhone 11 xa
Arko Hami sanga product Galaxy S21 xa ra yaslai Rs. 699.99 parcha.
Arko Hami sanga product Redmi Note 9 pro xa ra yaslai Rs. 999.99 parcha.



In [36]:
bot.process_message("Samsung ko galaxy s21 lai kati price ho")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': 'Samsung', 'Category': None, 'Quantity': 's21', 'Order_Number': None}


'Hami sanga product Rs. 699.99 parne Galaxy S21 xa\n'

In [37]:
bot.process_message("Apple ko kun product available xa")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': 'Apple', 'Category': None, 'Quantity': None, 'Order_Number': None}


'Hami sanga product Rs. 799.99 parne IPhone 11 xa\nArko Hami sanga product Macbook Pro xa ra yaslai Rs. 1299.99 parcha.\n'

In [38]:
bot.process_message("kun kun product available xa")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Kun product ko barema janna chahau huncha name, category wa brand vannu hola'

## Order New

In [39]:
bot = EcommerceBot(intent_model, entity_model)

In [40]:
a = ["a", "b"]
e = {"a": None, "b": 10}

[entity for entity in a if e[entity] is  None]

['a']

In [41]:
bot.process_message("malai saman order garnu xa")

Intent: order_product
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Kripaya pura details dinuhos: Product, Quantity, Location pani vannuhos.'

In [42]:
bot.process_message("malai Apple ko mobile 1 ta Dharan ma delivery garne")

Intent: order_product
Entities: {'Location': 'Dharan', 'Product': None, 'Brand': 'Apple', 'Category': 'mobile', 'Quantity': '1', 'Order_Number': None}


'Kripaya pura details dinuhos: Product pani vannuhos.'

In [43]:
bot.process_message("malai Apple Iphone 11 Dharan ma 2 ta deliver garne")

Intent: order_product
Entities: {'Location': 'Dharan', 'Product': 'Apple Iphone 11', 'Brand': None, 'Category': None, 'Quantity': '2', 'Order_Number': None}


'Tapaiko order confirm vayo Apple Iphone 11, total quantity:  2 wata ra delivery location Dharan total Cost Rs. 1599.98 hajur ko order number: ORD12345'

In [44]:
bot.process_message("mero order ORD12345 track garidinu")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': 'ORD12345'}


'Hajur ko order ko Processing hudei xa.'

In [45]:
bot.process_message("mero order Apple ko Iphone 3 ta Delivery location Dharan")

Intent: order_product
Entities: {'Location': 'Dharan', 'Product': 'Iphone', 'Brand': 'Delivery', 'Category': None, 'Quantity': '3', 'Order_Number': None}


'Tapaiko order confirm vayo Iphone, total quantity:  3 wata ra delivery location Dharan total Cost Rs. 2399.9700000000003 hajur ko order number: ORD12345'

## Testing the bot response

In [46]:
def chat_with_bot(question: str) -> str:
    response = bot.process_message(question)
    print(response)

In [47]:
chat_with_bot("malai saman order garnu thiyo")

Intent: order_product
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}
Kripaya pura details dinuhos: Product, Quantity, Location pani vannuhos.


In [48]:
chat_with_bot("redmi note 9 ko price kati ho")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': '9', 'Order_Number': None}
Kun product ko barema janna chahau huncha name, category wa brand vannu hola


In [52]:
chat_with_bot("kun kun mobile xa jankari dinu")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': 'mobile', 'Quantity': None, 'Order_Number': None}
[{'id': 1, 'name': 'IPhone 11', 'category': 'mobile', 'brand': 'Apple', 'price': 799.99}, {'id': 2, 'name': 'Galaxy S21', 'category': 'mobile', 'brand': 'Samsung', 'price': 699.99}, {'id': 6, 'name': 'Redmi Note 9 pro', 'category': 'mobile', 'brand': 'Xiaomi', 'price': 999.99}]
Hami sanga product Rs. 799.99 parne IPhone 11 xa
Arko Hami sanga product Galaxy S21 xa ra yaslai Rs. 699.99 parcha.
Arko Hami sanga product Redmi Note 9 pro xa ra yaslai Rs. 999.99 parcha.



In [51]:
bot.process_message("kun kun mobile xa jankari dinu")

Intent: product_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': 'mobile', 'Quantity': None, 'Order_Number': None}
[{'id': 1, 'name': 'IPhone 11', 'category': 'mobile', 'brand': 'Apple', 'price': 799.99}, {'id': 2, 'name': 'Galaxy S21', 'category': 'mobile', 'brand': 'Samsung', 'price': 699.99}, {'id': 6, 'name': 'Redmi Note 9 pro', 'category': 'mobile', 'brand': 'Xiaomi', 'price': 999.99}]


'Hami sanga product Rs. 799.99 parne IPhone 11 xa\nArko Hami sanga product Galaxy S21 xa ra yaslai Rs. 699.99 parcha.\nArko Hami sanga product Redmi Note 9 pro xa ra yaslai Rs. 999.99 parcha.\n'